In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from pprint import pformat

from hloc import extract_features, match_features, pairs_from_covisibility
from hloc import triangulation, localize_sfm, visualization
from hloc import retrieval, pairs_from_retrieval

# Pipeline for outdoor day-night visual localization

## Setup
Here we declare the paths to the dataset, the reconstruction and localization outputs, and we choose the feature extractor and the matcher. You only need to download the [Aachen Day-Night dataset](https://www.visuallocalization.net/datasets/) and put it in `datasets/aachen/`, or change the path.

In [ ]:
dataset = Path('datasets/Tutorial 01 Dataset/')
images = dataset / 'images/'


outputs = Path('outputs/sfm/')
sfm_pairs = outputs / 'pairs-exhaustive.txt'
loc_pairs = outputs / 'pairs-query-netvlad50.txt'

reference_sfm = outputs / 'sfm_superpoint+superglue'
triangulated_sfm = outputs / 'sfm_superpoint+superglue_triangulated'
results = outputs / 'Tutorial_hloc_superpoint+superglue_netvlad50.txt'

global_descriptor_name = "openibl_4096.h5"

In [ ]:
# list the standard configurations available
print(f'Configs for feature extractors:\n{pformat(extract_features.confs)}')
print(f'Configs for feature matchers:\n{pformat(match_features.confs)}')

In [ ]:
# pick one of the configurations for extraction and matching
# you can also simply write your own here!
feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

## Extract local features for database and query images

In [ ]:
feature_path = extract_features.main(feature_conf, images, outputs)

The function returns the path of the file in which all the extracted features are stored.

## Generate pairs for the SfM reconstruction
Instead of matching all database images exhaustively, we exploit the existing SIFT model to find which image pairs are the most covisible. We first convert the SIFT model from the NVM to the COLMAP format, and then do a covisiblity search, selecting the top 20 most covisibile neighbors for each image.

In [ ]:
pairs_from_covisibility.main(
    reference_sfm, sfm_pairs, num_matched=20)

## Match the database images

In [ ]:
sfm_match_path = match_features.main(matcher_conf, sfm_pairs, feature_conf['output'], outputs)

The function returns the path of the file in which all the computed matches are stored.

## Triangulate a new SfM model from the given poses
We triangulate the sparse 3D pointcloud given the matches and the reference poses stored in the SIFT COLMAP model.

In [ ]:
triangulation.main(
    triangulated_sfm,
    reference_sfm,
    images,
    sfm_pairs,
    feature_path,
    sfm_match_path,
    colmap_path='colmap')

## Match the query images
Here we assume that the localization pairs are already computed using image retrieval (NetVLAD). To generate new pairs from your own global descriptors, have a look at `hloc/pairs_from_retrieval.py`. These pairs are also used for the localization - see below.

In [ ]:
desc_path = retrieval.main(retrieval.confs["openibl"], images, outputs, global_descriptor_name)
pairs_from_retrieval.main(desc_path, loc_pairs, 20, db_prefix="d", query_prefix="q")
loc_match_path = match_features.main(matcher_conf, loc_pairs, feature_conf['output'], outputs)

## Localize!
Perform hierarchical localization using the precomputed retrieval and matches. The file `Aachen_hloc_superpoint+superglue_netvlad50.txt` will contain the estimated query poses. Have a look at `Aachen_hloc_superpoint+superglue_netvlad50.txt_logs.pkl` to analyze some statistics and find failure cases.

In [ ]:
localize_sfm.main(
    triangulated_sfm,
    dataset / 'queries_with_intrinsics.txt',
    loc_pairs,
    feature_path,
    loc_match_path,
    results,
    covisibility_clustering=False)  # not required with SuperPoint+SuperGlue

## Visualizing the SfM model
We visualize some of the database images with their detected keypoints.

Color the keypoints by track length: red keypoints are observed many times, blue keypoints few.

In [ ]:
visualization.visualize_sfm_2d(reference_sfm, images, n=1, color_by='track_length')

Color the keypoints by visibility: blue if sucessfully triangulated, red if never matched.

In [ ]:
visualization.visualize_sfm_2d(reference_sfm, images, n=1, color_by='visibility')

Color the keypoints by triangulated depth: red keypoints are far away, blue keypoints are closer.

In [ ]:
visualization.visualize_sfm_2d(reference_sfm, images, n=1, color_by='depth')

## Visualizing the localization
We parse the localization logs and for each query image plot matches and inliers with a few database images.

In [ ]:
visualization.visualize_loc(
    results, images, reference_sfm, n=1, top_k_db=1, prefix='query/night', seed=2)